In [12]:
import numpy as np
from control.matlab import *
import matplotlib.pyplot as plt

In [13]:
zc = 0.22
dt = 0.01
t_preview = 1
Qe = 1e-4
R = 1e-6

In [14]:
def get_preview_control_parameter(zc, dt, t_preview, Qe, R):
    g = 9.81
    A = np.matrix([[0, 1, 0],
                   [0, 0, 1],
                   [0, 0, 0]])
    B = np.matrix([[0],
                   [0],
                   [1]])
    C = np.matrix([[1, 0, -zc/g]])
    D = 0;
    
    sys_c = ss(A, B, C, D);            
    sys_d = c2d(sys_c, dt);
    [A_d, B_d, C_d, D_d] = ssdata(sys_d);

    C_d_dot_A_d = C_d*A_d

    C_d_dot_B_d = C_d*B_d

    A_tilde = np.matrix([[1, C_d_dot_A_d[0,0], C_d_dot_A_d[0,1], C_d_dot_A_d[0,2]],
                         [0, A_d[0,0], A_d[0,1], A_d[0,2]],
                         [0, A_d[1,0], A_d[1,1], A_d[1,2]],
                         [0, A_d[2,0], A_d[2,1], A_d[2,2]]])
    B_tilde = np.matrix([[C_d_dot_B_d[0,0]],
                         [B_d[0,0]],
                         [B_d[1,0]],
                         [B_d[2,0]]])
    C_tilde = np.matrix([[1, 0, 0, 0]])

    Q = np.matrix([[Qe, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0],
                   [0, 0, 0, 0]])

    P, _, K  = dare(A_tilde, B_tilde, Q, R);

    Gi = K[0,0:1]
    Gx = K[0,1:]

    N = np.arange(0,t_preview,dt).reshape(1,-1)

    Gd = np.zeros(N.shape)

    Gd[0,0] = -Gi

    Ac_tilde = A_tilde - B_tilde * K;

    I_tilde = np.matrix([[1],[0],[0],[0]])

    X_tilde = -Ac_tilde.T*P*I_tilde;

    for i in range(1, N.shape[1]):
        Gd[0,i] = (R+B_tilde.T*P*B_tilde)**(-1)*B_tilde.T*X_tilde;
        X_tilde = Ac_tilde.T*X_tilde;
        
    return np.array(A_d), np.array(B_d), np.array(C_d), np.array(Gi), np.array(Gx), Gd

In [15]:
A_d, B_d, C_d, Gi, Gx, Gd = get_preview_control_parameter(zc, dt, t_preview, Qe, R)

In [18]:
Gd

array([[ -9.23868228,  -9.25780946,  -9.29295039,  -9.34116283,
         -9.39967715,  -9.46589683,  -9.53739812,  -9.61192901,
         -9.68740739,  -9.76191866,  -9.83371265,  -9.90120012,
         -9.96294866, -10.01767828, -10.06425652, -10.1016933 ,
        -10.12913544, -10.14586098, -10.15127328, -10.14489495,
        -10.12636167, -10.09541596, -10.0519008 ,  -9.99575338,
         -9.92699868,  -9.84574324,  -9.75216889,  -9.64652663,
         -9.52913056,  -9.40035193,  -9.26061344,  -9.11038352,
         -8.95017094,  -8.78051952,  -8.60200302,  -8.41522032,
         -8.22079072,  -8.0193495 ,  -7.81154367,  -7.59802802,
         -7.37946131,  -7.15650272,  -6.92980857,  -6.70002919,
         -6.46780611,  -6.23376938,  -5.9985352 ,  -5.76270369,
         -5.52685694,  -5.29155722,  -5.0573454 ,  -4.82473961,
         -4.594234  ,  -4.3662978 ,  -4.14137443,  -3.91988089,
         -3.70220721,  -3.48871616,  -3.27974297,  -3.07559531,
         -2.87655334,  -2.68286986,  -2.